<a href="https://colab.research.google.com/github/morarez/dataset-recommender-system/blob/main/hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
import pandas as pd
import numpy as np
import glob
import os

Mounted at /content/gdrive


# Pre processing


In [ ]:
#df = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/Metadata_21june/metadata_v3.json')
df = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/ni_3900/metadata_ni_3900.json', orient='index')


In [ ]:
df["text"] = df["en_abstract"] + " " + df['en_title']

In [ ]:
df.shape

(3906, 6)

In [ ]:
df

,keyword,type,id,en_abstract,en_title,text
0,"[neuroimaging, workflow, pipeline, preprocessi...",software,50|doi_________::77d25c5587f0b1ab44b34485d4ecb37f,Hotfix Release <ul> <li>[FIX] Breadcrumb repor...,poldracklab/fmriprep: 1.2.5,Hotfix Release <ul> <li>[FIX] Breadcrumb repor...
1,"[neuroimaging, workflow, pipeline, preprocessi...",software,50|doi_________::9b509ec8729163a6ef7e1b907b775388,Release notes <p>This release is almost comple...,Poldracklab/Fmriprep: 1.0.0-Rc12,Release notes <p>This release is almost comple...
2,"[neuroimaging, workflow, pipeline, preprocessi...",software,50|doi_________::c1881ca8737b1dd96123c03e5f11e4c2,Release Notes <p>Updated ecosystem's versions ...,poldracklab/fmriprep: 1.3.1,Release Notes <p>Updated ecosystem's versions ...
3,"[neuroimaging, workflow, pipeline, preprocessi...",software,50|doi_________::db035d2dbe49ed7a1dc68f5163222c72,Release Notes <p>Several bug fixes. With thank...,Poldracklab/Fmriprep: 1.2.2,Release Notes <p>Several bug fixes. With thank...
4,"[neuroimaging, workflow, pipeline, statistics,...",software,50|doi_________::b72d03a0b3d6a87011d520606135f540,Release Notes <p>New feature release in the 0....,poldracklab/fitlins: 0.7.0,Release Notes <p>New feature release in the 0....
...,...,...,...,...,...,...
3930,"[multi-source learning, kernel methods, neuroi...",publication,50|dedup_wf_001::eba45041f386b8955a152f9ad92a0d75,Inter-subject learning is a family of learning...,A multi-source perspective on inter-subject le...,Inter-subject learning is a family of learning...
3931,"[Endocrinology, Diabetes and Metabolism, Clini...",publication,50|doi_dedup___::39b99a1f8b30cda3d87e24a6bd80af41,Impulsivity refers to a tendency to act rapidl...,Overlapping Neural Endophenotypes in Addiction...,Impulsivity refers to a tendency to act rapidl...
3932,"[Adult, Analysis of Variance, Body Mass Index,...",publication,50|doi_dedup___::7f5e86c341502b8113f7f6b4163ef38f,Background/Objective The brain plays a central...,Sex differences in the influence of body mass ...,Background/Objective The brain plays a central...
3933,"[ddc:500, Multidisciplinary, Magnetic resonanc...",publication,50|doi_dedup___::22cbf08893400f59f38e5145dde6aa70,The relatively high imaging speed of EPI has l...,Whole-brain high in-plane resolution fMRI usin...,The relatively high imaging speed of EPI has l...


In [ ]:
df.reset_index(drop=True, inplace=True)


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(lemma_words)


df['cleanText']=df['text'].map(lambda s:preprocess(s)) 

In [ ]:
data = df['cleanText']

In [ ]:
pip install Gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# users

In [ ]:
users_d = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/ni_3900/users_ni_3900.json',orient='index')

In [ ]:
def d_counter(user_prof):
  d_counter = 0
  for id in user_prof:
    item_type = df[df['id'] == id]['type']
    for i in item_type:
      if str(i) == 'dataset':
        d_counter += 1
  return d_counter

In [ ]:
users_d['d_counter'] = users_d['id'].apply(lambda row : d_counter(row))

In [ ]:
users_d = users_d[users_d['d_counter'] != 0 ]
users_d = users_d[users_d['d_counter'] != 1 ]
users_d = users_d[users_d['d_counter'] != 2 ]
users_d = users_d[users_d['d_counter'] != 3 ]

users_d.reset_index(drop=True, inplace=True)

## Leave one out

In [ ]:
# ## IMPORTANT: if leave-2-out/leave-3-out has already been run, you need to load users from the start

def leave_one_out_d(profile_items):
  for item in profile_items:
      item_type = df[df['id'] == item]['type'].tolist()
      if item_type[0] == 'dataset':
        profile_items.remove(item)
        return item

In [ ]:
users_d['leave_one_out_d'] = users_d['id'].apply(lambda x:leave_one_out_d(x))

## Leave two out


In [ ]:
# ## IMPORTANT: if leave-1-out/leave-2-out has already been run, you need to load users from the start

def leave_two_out_d(profile_items):
  dataset_list = []
  for item in profile_items:
      item_type = df[df['id'] == item]['type'].tolist()
      if item_type[0] == 'dataset':
          dataset_list.append(item)
          if len(dataset_list)==2:
            for ds in dataset_list:
                profile_items.remove(ds)

            return dataset_list

In [ ]:
users_d['leave_two_out_d'] = users_d['id'].apply(lambda x:leave_two_out_d(x))

## Leave three out

In [ ]:
# ## IMPORTANT: if leave-1-out/leave-2-out has already been run, you need to load users from the start

def leave_three_out_d(profile_items):
  dataset_list = []
  for item in profile_items:
      item_type = df[df['id'] == item]['type'].tolist()
      if item_type[0] == 'dataset':
          dataset_list.append(item)
          if len(dataset_list)==3:
            for ds in dataset_list:
                profile_items.remove(ds)

            return dataset_list

In [ ]:
users_d['leave_three_out_d'] = users_d['id'].apply(lambda x:leave_three_out_d(x))

# Doc2Vec 

## Train model

In [ ]:
from gensim import models
def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield models.doc2vec.TaggedDocument(list_of_words, [i])
data_training = list(tagged_document(data))

In [ ]:
model = models.doc2vec.Doc2Vec(vector_size=256, min_count=2, epochs=100)
model.build_vocab(data_training)
model.train(data_training, total_examples=model.corpus_count, epochs=model.epochs)


## Creating user D2V vector

In [ ]:
def user_vector_doc2vec(user_list):
  import numpy as np
  user_profile = []
  for user_id in user_list:
    user_index = df[df['id']==user_id].index
    user_profile.append(model.docvecs[user_index[0]])
  column_average = [sum(sub_list) / len(sub_list) for sub_list in zip(*user_profile)]
  user_vector = np.array(column_average)
  return user_vector

In [ ]:
users_d['user_vector_doc2vec'] = users_d['id'].apply(lambda s:user_vector_doc2vec(s)) 

# Node2Vec 

In [ ]:
#all_files = glob.glob("/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/relation_jun21/*.json")
#relation = pd.concat((pd.read_json(f,lines=True) for f in all_files))
#relation.drop_duplicates(inplace=True)
#relation.reset_index(drop=True, inplace=True)

In [ ]:
relation = pd.read_json("/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/data_with_relations_v1/relations.json",lines=True)

In [ ]:
relation.drop_duplicates(inplace=True)
relation.reset_index(drop=True, inplace=True)

In [ ]:
pip install networkx==2.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Created wheel for networkx: filename=networkx-2.3-py2.py3-none-any.whl size=1556008 sha256=fba5aebdd7a779cb88963bcbb9e6aafbb821a0255679fcd5d547a1f0b4f4b60f
  Stored in directory: /root/.cache/pip/wheels/44/e6/b8/4efaab31158e9e9ca9ed80b11f6b11130bac9a9672b3cbbeaf
Successfully built networkx
  Attempting uninstall: networkx
    Found existing installation: networkx 2.6.3
    Uninstalling networkx-2.6.3:
      Successfully uninstalled networkx-2.6.3


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
# Build your graph
G=nx.from_pandas_edgelist(relation, 'source', 'target')

giant = max(nx.connected_component_subgraphs(G), key=len)
nodes_list = list(giant.nodes) #List of nodes

In [ ]:
##### counting artifacts #####
pubs = 0
not_found = 0
for item in nodes_list:
  item_type = df[df['id'] == item]['type'].tolist()
  try:
    if item_type[0] == 'software':
      pubs = pubs + 1
  except:
    giant.remove_node(item)
    #print(item)
pubs

77

In [ ]:
nx.info(giant)

'Name: \nType: Graph\nNumber of nodes: 3906\nNumber of edges: 4043\nAverage degree:   2.0701'

In [ ]:
authors = users_d.explode('id')
users_g=nx.from_pandas_edgelist(authors, 'author_name', 'id')
G = nx.compose(giant, users_g)

In [ ]:
pip install node2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for node2vec: filename=node2vec-0.4.3-py3-none-any.whl size=5980 sha256=edb71a8e6f65668c08ba42cf584ec4669f2dc3c9c7777f1669e8a0a8111a0e43
  Stored in directory: /root/.cache/pip/wheels/07/62/78/5202cb8c03cbf1593b48a8a442fca8ceec2a8c80e22318bae9
Successfully built node2vec


In [ ]:
from node2vec import Node2Vec as n2v
g_emb = n2v(G, dimensions=256)

WINDOW = 4 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 64 # Node2Vec batch words

mdl = g_emb.fit(
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

# create embeddings dataframe
emb_df = (
    pd.DataFrame(
        [mdl.wv.get_vector(str(n)) for n in G.nodes()],
        index = G.nodes
    )
)

emb_df.head()

Computing transition probabilities:   0%|          | 0/4059 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [03:48<00:00, 22.83s/it]


,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
50|doi_dedup___::3da4b5b1a22214ecc1bb0eb03295eab0,-0.005207,0.110791,-0.185311,0.053689,0.222624,-0.068528,0.006292,-0.093620,0.577514,0.178990,...,0.142705,-0.254240,-0.476120,0.091133,0.049805,-0.039526,0.033121,0.256799,0.038865,0.140505
50|doi_dedup___::4d029bd3cd0928f8115d44185ee2a708,0.073171,0.241159,0.246454,0.144039,0.370231,-0.393657,-0.574680,-0.103913,-0.226758,0.565594,...,-0.247299,-0.511251,-0.124770,0.241725,0.277702,-0.232385,-0.228270,0.170603,0.113733,-0.545246
50|doi_dedup___::8d220e65a7dc941055151965a7f7d80c,0.016064,-0.200839,-0.011821,0.205817,0.229148,0.011544,-0.003320,-0.235874,0.710456,0.592544,...,-0.368819,0.209103,0.004926,-0.145881,-0.005403,0.218889,0.138002,-0.023144,-0.226346,-0.159449
50|arXiv_______::f89f361dc90195bdf0a737fe193d7db8,-0.013083,0.241464,0.254438,0.208799,0.029638,-0.160771,-0.156133,0.106699,0.194448,-0.056061,...,0.066328,-0.245797,-0.361530,0.371413,-0.387059,-0.451244,-0.457428,0.026024,0.234617,0.398862
50|doi_dedup___::528fe145915ceda3e385c1907fb25a7b,0.073061,-0.044495,-0.041462,0.112776,0.078548,-0.116866,-0.360024,-0.095010,0.266857,0.426064,...,0.189452,-0.339363,0.227647,0.178742,-0.159706,-0.242945,-0.082716,0.402456,0.042787,-0.514669


In [ ]:
dataset = df.loc[(df['type']=='dataset')]

In [ ]:
def predict_links(df, article_id):
    '''
    This function will predict the top N links a node (article_id) should be connected with
    which it is not already connected with in G.
    
    params:
        G (Netowrkx Graph) : The network used to create the embeddings
        df (DataFrame) : The dataframe which has embeddings associated to each node
        article_id (Integer) : The article you're interested 
        N (Integer) : The number of recommended links you want to return
        
    returns:
        This function will return a list of nodes the input node should be connected with.
    '''
    
    # separate target article with all others
    article = df[df.index == article_id]
    
    return np.array(article)[0]
    
    
  
#predict_links(G = G, df = emb_df, article_id = "Goeree, Ron", N = 10)

## Creating node2vec vector

In [ ]:
def user_vector_node2vec(name):
  user_vector = predict_links(df = emb_df, article_id = name)
  return user_vector

In [ ]:
users_d['user_vector_node2vec'] = users_d['author_name'].apply(lambda x:user_vector_node2vec(x))

## generate average vector

In [ ]:
users_d['average_vector'] = (users_d['user_vector_doc2vec'] + users_d['user_vector_node2vec'])/2

# Artefacts # run it after the algorithms 

In [ ]:
def item_vector_node2vec(name):
  item_vector = predict_links(df = emb_df, article_id = name)
  return item_vector

In [ ]:
df['item_vector_node2vec'] = df['id'].apply(lambda x:item_vector_node2vec(x))

In [ ]:
## Doc2Vec ##

In [ ]:
df['item_vector_doc2vec'] = pd.np.empty((len(df), 0)).tolist()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.


In [ ]:
for i,vec in enumerate(model.docvecs):
    df.at[i, "item_vector_doc2vec"] = vec

### IGNORE ERROR

KeyError: ignored

In [ ]:
df['average_vector'] = (df['item_vector_doc2vec'] + df['item_vector_node2vec'])/2


# Similarity - Dataset Recsys

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def reshape_func(vector):
  return np.array(vector.reshape(1,-1), dtype=object)

In [ ]:
df['average_vector_reshaped'] = df['average_vector'].apply(lambda x:reshape_func(x))

In [ ]:
users_d['average_vector_reshaped'] = users_d['average_vector'].apply(lambda x:reshape_func(x))

In [ ]:
def recommend_hybrid(user):
  sim_list = []
  for vec in df['average_vector_reshaped']:
    sim_list.append(cosine_similarity(user['average_vector_reshaped'], vec)[0][0])
  idx = df['average_vector_reshaped'].index.tolist()
  idx_sim = dict(zip(idx, sim_list))
  idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)
  similar_items = idx_sim[:100]
  all_items = [df['id'][art[0]] for art in similar_items]
  filtered_items = [n for n in all_items if n not in list(user['id'])]
  return filtered_items[:10]

In [ ]:
users_d['rec_items'] = users_d.apply(lambda x:recommend_hybrid(x),axis=1)

### Hit-rate@1 Evaluation

In [ ]:
def hit_rate_d(row):
  for item in row["rec_items"]:
    if item == row['leave_one_out_d']:
      return 1
  return 0 

In [ ]:
users_d['hit_rate_d'] = users_d.apply(lambda x:hit_rate_d(x),axis=1)

In [ ]:
users_d['hit_rate_d'].mean()

0.6993464052287581

### Hit-rate@2 Evaluation

In [ ]:
def hit_rate_d(row):
  for item in row["rec_items"]:
    if item in row['leave_two_out_d']:
      return 1
  return 0 

In [ ]:
users_d['hit_rate_d'] = users_d.apply(lambda x:hit_rate_d(x),axis=1)

In [ ]:
users_d['hit_rate_d'].mean()

0.8496732026143791

### Hit-rate@3 Evaluation

In [ ]:
users_d[users_d['leave_three_out_d'].isna()]

,author_name,id,d_counter,leave_three_out_d,user_vector_doc2vec,user_vector_node2vec,average_vector,average_vector_reshaped,rec_items


In [ ]:
def hit_rate_d(row):
  for item in row["rec_items"]:
    if item in row['leave_three_out_d']:
      return 1
  return 0 

In [ ]:
users_d['hit_rate_d'] = users_d.apply(lambda x:hit_rate_d(x),axis=1)

In [ ]:
users_d['hit_rate_d'].mean()

0.8562091503267973

# Doc2Vec - Software Recsys

In [ ]:
from gensim import models
def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield models.doc2vec.TaggedDocument(list_of_words, [i])
data_training = list(tagged_document(data))

In [ ]:
model = models.doc2vec.Doc2Vec(vector_size=64, min_count=2, epochs=100)
model.build_vocab(data_training)
model.train(data_training, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
def user_vector_doc2vec(user_list):
  import numpy as np
  user_profile = []
  for user_id in user_list:
    user_index = df[df['id']==user_id].index
    user_profile.append(model.docvecs[user_index[0]])
  column_average = [sum(sub_list) / len(sub_list) for sub_list in zip(*user_profile)]
  user_vector = np.array(column_average)
  return user_vector


In [ ]:
users = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/users_june21/users_v3.json')
users = users.loc[users.astype(str).drop_duplicates(subset=['id']).index]
users.reset_index(drop=True, inplace=True)

In [ ]:
def sw_counter(user_prof):
  sw_counter = 0
  for id in user_prof:
    item_type = df[df['id'] == id]['type']
    for i in item_type:
      if str(i) == 'software':
        sw_counter += 1
  return sw_counter

In [ ]:
users['sw_counter'] = users['id'].apply(lambda row : sw_counter(row))

In [ ]:
users_sw = users[users['sw_counter'] != 0 ]
users_sw.reset_index(drop=True, inplace=True)

In [ ]:
def leave_one_cv_sw(profile_items):
   for item in profile_items:
      item_type = df[df['id'] == item]['type'].tolist()
      if item_type[0] == 'software':
        users_sw['id'] = users_sw['id'].apply(lambda x:[i for i in x if i != item])
        return item
      

In [ ]:
users_sw['leave_one_cv_sw'] = users_sw['id'].apply(lambda x:leave_one_cv_sw(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
users_sw['user_vector_doc2vec'] = users_sw['id'].apply(lambda s:user_vector_doc2vec(s)) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Node2Vec - Software RecSys

In [ ]:
all_files = glob.glob("/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/relation_jun21/*.json")
relation = pd.concat((pd.read_json(f,lines=True) for f in all_files))
relation.drop_duplicates(inplace=True)
relation.reset_index(drop=True, inplace=True)

In [ ]:
pip install networkx==2.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
# Build your graph
G=nx.from_pandas_edgelist(relation, 'source', 'target')

giant = max(nx.connected_component_subgraphs(G), key=len)
nodes_list = list(giant.nodes) #List of nodes

In [ ]:
authors = users_sw.explode('id')
users_g=nx.from_pandas_edgelist(authors, 'author_name', 'id')
G = nx.compose(giant, users_g)

In [ ]:
pip install node2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from node2vec import Node2Vec as n2v
g_emb = n2v(G, dimensions=64)

WINDOW = 4 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 64 # Node2Vec batch words

mdl = g_emb.fit(
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

# create embeddings dataframe
emb_df = (
    pd.DataFrame(
        [mdl.wv.get_vector(str(n)) for n in G.nodes()],
        index = G.nodes
    )
)

emb_df.head()

Computing transition probabilities:   0%|          | 0/2447 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [01:47<00:00, 10.78s/it]


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
50|doi_dedup___::b5c0187b9b14a1005fc252cca6dfd63a,1.202392,-0.557133,-0.528513,0.154287,0.435833,0.525865,0.167366,0.145525,1.847796,0.233889,...,0.581680,1.095658,0.330313,-0.197624,-0.362270,0.692087,0.521978,-0.868873,0.977962,0.100116
50|doi_dedup___::1436daf2f6408384a1de58a7ae5b8619,0.722363,0.069371,0.213405,0.552462,0.642898,-0.282088,-0.096145,-0.462140,0.365763,-0.186876,...,-0.098001,0.596585,-0.737817,0.462870,0.194390,0.461448,-0.274632,-0.091218,0.794302,-0.738732
50|doi_dedup___::4e4fcf4c5658febfbe65d08a65905342,0.522479,-0.043646,-0.607593,0.585873,0.468629,0.960085,0.481845,0.263597,1.094236,0.745757,...,-0.524652,0.219186,-0.213119,-0.281225,0.130193,0.693890,0.254202,-0.221391,0.028132,-0.891682
50|doi_dedup___::c2c40984b2fba1e242bc06bcd0bb91cd,-0.048725,-0.259533,0.470435,0.945771,0.339599,0.035982,-0.526796,-0.289019,0.637828,0.774852,...,0.504572,0.082771,-0.130482,0.904876,-0.467881,0.109620,0.174369,-0.310916,0.688191,-0.516854
50|doi_dedup___::1d36dcb6c376a2fecef8ec7ea7f5e78c,0.924456,-0.305343,0.305293,0.622434,1.088523,0.227392,0.325939,0.216133,0.944095,-0.208833,...,-0.748517,1.402469,0.015325,0.761842,0.338157,0.345113,-0.163361,0.136774,-0.140990,0.157410


In [ ]:
software = df.loc[(df['type']=='software')]

In [ ]:
def predict_links(df, article_id):
    '''
    This function will predict the top N links a node (article_id) should be connected with
    which it is not already connected with in G.
    
    params:
        G (Netowrkx Graph) : The network used to create the embeddings
        df (DataFrame) : The dataframe which has embeddings associated to each node
        article_id (Integer) : The article you're interested 
        N (Integer) : The number of recommended links you want to return
        
    returns:
        This function will return a list of nodes the input node should be connected with.
    '''
    
    # separate target article with all others
    article = df[df.index == article_id]
    
    return np.array(article)[0]
    
    
  
#predict_links(G = G, df = emb_df, article_id = "Goeree, Ron", N = 10)

In [ ]:
def user_vector_node2vec(name):
  user_vector = predict_links(df = emb_df, article_id = name)
  return user_vector

In [ ]:
users_sw['user_vector_node2vec'] = users_sw['author_name'].apply(lambda x:user_vector_node2vec(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
users_sw['average_vector'] = (users_sw['user_vector_doc2vec'] + users_sw['user_vector_node2vec'])/2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.



# Similarity - Software RecSys

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def reshape_func(vector):
  return np.array(vector.reshape(1,-1), dtype=object)

In [ ]:
df['average_vector_reshaped'] = df['average_vector'].apply(lambda x:reshape_func(x))

In [ ]:
users_sw['average_vector_reshaped'] = users_sw['average_vector'].apply(lambda x:reshape_func(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
print(type(df['average_vector_reshaped'][0]))

<class 'numpy.ndarray'>


In [ ]:
def recommend_hybrid(user):
  sim_list = []
  for vec in df['average_vector_reshaped']:
    sim_list.append(cosine_similarity(user['average_vector_reshaped'], vec)[0][0])
  idx = df['average_vector_reshaped'].index.tolist()
  idx_sim = dict(zip(idx, sim_list))
  idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)
  similar_items = idx_sim[:100]
  all_items = [df['id'][art[0]] for art in similar_items]
  filtered_items = [n for n in all_items if n not in list(user['id'])]
  return filtered_items[:10]

In [ ]:
users_sw['rec_items'] = users_sw.apply(lambda x:recommend_hybrid(x),axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
def hit_rate_sw(row):
  for item in row["rec_items"]:
    if item == row['leave_one_cv_sw']:
      return 1
  return 0 

In [ ]:
users_sw['hit_rate_sw'] = users_sw.apply(lambda x:hit_rate_sw(x),axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
users_sw['hit_rate_sw'].mean()

0.41935483870967744